In [1]:
import pandas as pd

d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d, index=[0,1])
d1 = {'col1': [3, 3], 'col2': [3, 4]}
df1 = pd.DataFrame(data=d1, index=[2,3])

In [9]:
combined = pd.concat([df, df1])

In [10]:
train_df = combined.sample(2)
val_df = combined.drop(train_df.index)

In [11]:
train_df

,col1,col2
2,3,3
0,1,3


In [12]:
val_df

,col1,col2
1,2,4
3,3,4


In [13]:
pd.concat([train_df, val_df
        ])

,col1,col2
2,3,3
0,1,3
1,2,4
3,3,4


In [1]:
import os
for configsfile in os.listdir(os.path.join(os.getcwd(), 'configs_long_gridsearch/')):
    print(configsfile)

configs1.json
configs2.json


#### How many images can we test on when not using augmented images, per min palsa?

In [ ]:
def filter_dataset(
    labels_file, augment, min_palsa_positive_samples, low_pals_in_val, n_samples):

    n_train = int(round(n_samples*0.8))
    n_val = int(round(n_samples*0.2))

    # labels as dataframe
    labels_df = pd.read_csv(labels_file, index_col=0)

    # remove augmented images depending on configs
    if not augment:
        labels_df = labels_df.loc[~labels_df.index.str.endswith('_aug')]

    # impose minimum palsa percentage for positive samples
    if min_palsa_positive_samples > 0:

        # find indices of samples with 0<x<threshold palsa
        drop_range = labels_df[
            (labels_df['palsa_percentage'] > 0)
            & (labels_df['palsa_percentage'] <= min_palsa_positive_samples)].index

        # remove low palsa images from train set
        train_df = labels_df.drop(drop_range).sample(n_train)

        # sample val images from unfiltered df
        if low_pals_in_val:
            val_df = labels_df.drop(train_df.index).sample(n_val)

        # sample val images from filtered df
        if not low_pals_in_val:
            val_df = labels_df.drop(drop_range)
            val_df = val_df.drop(train_df.index).sample(n_val)

    # if no minimum palsa percentage is imposed
    elif min_palsa_positive_samples == 0:
        train_df = labels_df.sample(n_train)
        val_df = labels_df.drop(train_df.index).sample(n_val)

    return train_df, val_df

In [1]:
import pandas as pd
labels_file = '/home/nadjaflechner/Palsa_data/generated_datasets/FINAL_200m_dataset/palsa_labels.csv'

In [2]:
labels_df = pd.read_csv(labels_file, index_col=0)
labels_df = labels_df.loc[~labels_df.index.str.endswith('_aug')]


In [3]:
labels_df

,palsa_percentage
760_73_5025_2018_crop_14,7.25
760_73_5025_2018_crop_15,0.75
760_73_5025_2018_crop_16,3.25
760_73_5025_2018_crop_17,1.50
760_73_5025_2018_crop_18,7.50
...,...
763_72_2550_2018_negcrop_6,0.00
763_72_2550_2018_negcrop_7,0.00
763_72_2550_2018_negcrop_8,0.00
763_72_2550_2018_negcrop_9,0.00


In [7]:
drop_range = labels_df[
    (labels_df['palsa_percentage'] > 0)
    & (labels_df['palsa_percentage'] <= 10)].index

In [8]:
len(drop_range)

4578

In [10]:
15164 - 4578

10586